<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Let's-try-to-extract-the-list-of-bus-destinations" data-toc-modified-id="Let's-try-to-extract-the-list-of-bus-destinations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Let's try to extract the list of bus destinations</a></span></li><li><span><a href="#Now,-we-should-find-GPS-coordinates" data-toc-modified-id="Now,-we-should-find-GPS-coordinates-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Now, we should find GPS coordinates</a></span></li></ul></div>

# Beogradska autobuska stancija

### Let's try to extract the list of bus destinations

In [1]:
import pandas as pd
!pip install folium
import folium
!pip install lxml
from lxml import etree, objectify
import xml.dom.minidom

In [2]:
doc = xml.dom.minidom.parse('destination.xml')
elements = doc.getElementsByTagName('DestinationDto')
for elem in elements:
    print(dir(elem))
    print (elem.firstChild.getElementsByTagNameNS)
    break

['ATTRIBUTE_NODE', 'CDATA_SECTION_NODE', 'COMMENT_NODE', 'DOCUMENT_FRAGMENT_NODE', 'DOCUMENT_NODE', 'DOCUMENT_TYPE_NODE', 'ELEMENT_NODE', 'ENTITY_NODE', 'ENTITY_REFERENCE_NODE', 'NOTATION_NODE', 'PROCESSING_INSTRUCTION_NODE', 'TEXT_NODE', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_attrs', '_attrsNS', '_call_user_data_handler', '_child_node_types', '_ensure_attributes', '_get_attributes', '_get_childNodes', '_get_firstChild', '_get_lastChild', '_get_localName', '_get_tagName', '_localName', '_magic_id_nodes', 'appendChild', 'attributes', 'childNodes', 'cloneNode', 'firstChild', 'getAttribute', 'getAttributeNS', 'getAttributeNode', 'getAttri

In [3]:
with open('destination.xml', encoding="utf-8") as fobj:
    xml = fobj.read()
root = etree.fromstring(xml)
print(type(root))
print(root[0])
print(root.getchildren())

<class 'lxml.etree._Element'>
<Element {http://schemas.datacontract.org/2004/07/WebApiService.Models}result at 0x2396a38cf00>
[<Element {http://schemas.datacontract.org/2004/07/WebApiService.Models}result at 0x2396a38f080>]


In [16]:
a = root.findall("{http://schemas.datacontract.org/2004/07/WebApiService.Models}result//{http://schemas.datacontract.org/2004/07/WebApiService.Models}DestinationDto")
destinations = {}

for elem in a:
    num = elem.getchildren()[0].text
    city = elem.getchildren()[1].text
    if num in destinations:
        continue
    else:
        destinations[num] = city



In [5]:
stations = pd.Series(destinations.values(), name="city").sort_values(ignore_index=True)
def lowercase(item):
    return item.lower()
stations = stations.apply(lowercase)
stations

0               13. oktobra p
1                     a.rampa
2                     ada  as
3       ada železnička stanic
4             aerodrom sombor
                ...          
2359          žujovića zabran
2360                žukovac r
2361                 županjac
2362                      žur
2363                žuta bara
Name: city, Length: 2364, dtype: object

Done! We've got a list of bus destinations from Belgrade

In [6]:
countries= ['Аустрија', 'Белгија', 'Босна и Херцеговина', 'Бугарска', 'Грчка', 'Данска', 'Италија', 'Македонија', 'Мађарска', 'Немачка', 'Република Српска', 'Словачка', 'Словенија', 'Србија', 'Турска', 'Француска', 'Холандија', 'Хрватска', 'Црна Гора', 'Чешка', 'Швајцарска','Шведска']
len(countries)

22

There are 22 countries in this list, let's start from Serbia

### Now, we should find GPS coordinates

 of those cities to pop them on the map. Fortunately, there is a website: https://simplemaps.com/data/rs-cities

In [7]:
coords = pd.read_csv('rs.csv')
# only 178 cities
cols = ['city', 'lat', 'lng']
coords = coords[cols]
coords['city'] = coords['city'].apply(lowercase) 
coords = coords.sort_values(by=['city'], ignore_index=True)
cities = coords['city']
cities

0                ada
1      aleksandrovac
2          aleksinac
3           alibunar
4             apatin
           ...      
173           žabalj
174           žabari
175         žagubica
176          žitište
177         žitorađa
Name: city, Length: 178, dtype: object

In [8]:
# now we should write a cicle before merging
for city in cities:
    for i in range(len(stations)):
        if stations[i].find(city) > -1:
            stations[i] = city
            break

stations

0               13. oktobra p
1                     a.rampa
2                         ada
3       ada železnička stanic
4                         bor
                ...          
2359          žujovića zabran
2360                žukovac r
2361                 županjac
2362                      žur
2363                žuta bara
Name: city, Length: 2364, dtype: object

In [9]:
stations_coords = stations.to_frame().merge(coords, on='city', how='inner')
stations_coords

,city,lat,lng
0,ada,45.8014,20.1222
1,bor,44.1303,22.1036
2,aleksandrovac,43.4553,21.0514
3,niš,43.3192,21.8961
4,aleksinac,43.5383,21.7047
...,...,...,...
168,šid,45.1283,19.2264
169,žabalj,45.3667,20.0667
170,žabari,44.3562,21.2143
171,žitište,45.4850,20.5497


In [21]:
#creating a map
m = folium.Map(
    location=[44.5, 20.3],
    zoom_start=8.5)

In [22]:
for _, city in stations_coords.iterrows():
    folium.Marker(
        location=[city['lat'], city['lng']],
        tooltip=city['city'],
        icon = folium.Icon(icon='circle')).add_to(m)
    
m

Plan:
1. Add markers in other countries
2. Add wikipedia link for every city